In [1]:
# 📌 Grad-CAM Visualization Script for Baseline Multi-Task Model

import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2

# 📁 Paths
model_path = "/home/srivatsa/Multiple_Crop_Disease_Detection/Models/Base_Line_Model/baseline_model_split.keras"
output_dir = "/home/srivatsa/Multiple_Crop_Disease_Detection/reports/Baseline_Model/Plots/GradCAM/"
os.makedirs(output_dir, exist_ok=True)

# 📌 Load Trained Model
model = tf.keras.models.load_model(model_path,compile=False)

# 📌 Get Last Conv Layer
last_conv_layer_name = "top_conv"  # EfficientNetV2B0 final conv block

# 🔍 Grad-CAM Function
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None, output_name="crop_output"):
    grad_model = tf.keras.models.Model(
        [model.inputs],
        [model.get_layer(last_conv_layer_name).output, model.get_layer(output_name).output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]

    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

# 📌 Overlay Heatmap
def overlay_heatmap(img, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    heatmap_resized = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap_resized), colormap)
    overlayed = heatmap_color * alpha + img
    return np.uint8(overlayed)

# 📌 Load Sample Images from Test TFRecord
feature_description = {
    "image": tf.io.FixedLenFeature([], tf.string),
    "crop": tf.io.FixedLenFeature([], tf.int64),
    "disease": tf.io.FixedLenFeature([], tf.int64),
}

def parse_example(example_proto):
    parsed = tf.io.parse_single_example(example_proto, feature_description)
    image = tf.image.decode_jpeg(parsed["image"], channels=3)
    image = tf.image.resize(image, [224, 224])
    image = tf.cast(image, tf.float32) / 255.0
    return image, parsed["crop"], parsed["disease"]

# 📂 TFRecord
test_tfrecord = "/home/srivatsa/Multiple_Crop_Disease_Detection/Dataset/PlantVillage_Structured/AUG_TFRecord/split/test.tfrecord"
test_dataset = tf.data.TFRecordDataset(test_tfrecord).map(parse_example).batch(1)

# 🔍 Visualize Grad-CAM for Few Samples
num_samples = 10
for i, (image, crop_label, disease_label) in enumerate(test_dataset.take(num_samples)):
    img_np = image[0].numpy()
    img_batch = tf.expand_dims(image[0], axis=0)

    # Grad-CAM Heatmap (Disease)
    heatmap_disease = make_gradcam_heatmap(img_batch, model, last_conv_layer_name, output_name="disease_output")
    overlay_disease = overlay_heatmap((img_np * 255).astype(np.uint8), heatmap_disease)

    # Grad-CAM Heatmap (Crop)
    heatmap_crop = make_gradcam_heatmap(img_batch, model, last_conv_layer_name, output_name="crop_output")
    overlay_crop = overlay_heatmap((img_np * 255).astype(np.uint8), heatmap_crop)

    # 📸 Save images
    plt.imsave(f"{output_dir}/GradCAM_Disease_{i}.png", overlay_disease)
    plt.imsave(f"{output_dir}/GradCAM_Crop_{i}.png", overlay_crop)

    print(f"Saved: GradCAM_Crop_{i}.png & GradCAM_Disease_{i}.png")

print("✅ Grad-CAM Visualization Completed!")

# 📌 Save Class Mapping
crop_mapping = {0: "Grape", 1: "Apple", 2: "Bell_Pepper", 3: "Corn", 4: "Strawberry",
                5: "Tomato", 6: "Cherry", 7: "Peach", 8: "Potato"}

disease_mapping = {
    0: "Grape___Leaf_blight", 1: "Grape___Esca", 2: "Grape___Black_rot", 3: "Grape___healthy",
    4: "Apple___Black_rot", 5: "Apple___Cedar_apple_rust", 6: "Apple___Apple_scab", 7: "Apple___healthy",
    8: "Bell_Pepper___Bacterial_spot", 9: "Bell_Pepper___healthy", 10: "Corn___Common_rust",
    11: "Corn___Cercospora_leaf_spot", 12: "Corn___Northern_Leaf_Blight", 13: "Corn___healthy",
    14: "Strawberry___healthy", 15: "Strawberry___Leaf_scorch", 16: "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    17: "Tomato___Bacterial_spot", 18: "Tomato___Target_Spot", 19: "Tomato___Leaf_Mold",
    20: "Tomato___Septoria_leaf_spot", 21: "Tomato___Tomato_mosaic_virus", 22: "Tomato___healthy",
    23: "Tomato___Early_blight", 24: "Tomato___Late_blight", 25: "Tomato___Spider_mites",
    26: "Cherry___Powdery_mildew", 27: "Cherry___healthy", 28: "Peach___Bacterial_spot",
    29: "Peach___healthy", 30: "Potato___healthy", 31: "Potato___Late_blight", 32: "Potato___Early_blight"
}

with open(f"{output_dir}/class_mapping.txt", "w") as f:
    f.write("🌿 Crop Class Mapping:\n")
    for k, v in crop_mapping.items():
        f.write(f"{k}: {v}\n")
    f.write("\n🦠 Disease Class Mapping:\n")
    for k, v in disease_mapping.items():
        f.write(f"{k}: {v}\n")

print("✅ Class mapping file saved at:", f"{output_dir}/class_mapping.txt")


2025-03-24 18:29:10.334295: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-24 18:29:10.493879: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742821150.567643    6420 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742821150.583386    6420 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-24 18:29:10.718748: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Saved: GradCAM_Crop_0.png & GradCAM_Disease_0.png
Saved: GradCAM_Crop_1.png & GradCAM_Disease_1.png
Saved: GradCAM_Crop_2.png & GradCAM_Disease_2.png
Saved: GradCAM_Crop_3.png & GradCAM_Disease_3.png
Saved: GradCAM_Crop_4.png & GradCAM_Disease_4.png
Saved: GradCAM_Crop_5.png & GradCAM_Disease_5.png
Saved: GradCAM_Crop_6.png & GradCAM_Disease_6.png
Saved: GradCAM_Crop_7.png & GradCAM_Disease_7.png
Saved: GradCAM_Crop_8.png & GradCAM_Disease_8.png
Saved: GradCAM_Crop_9.png & GradCAM_Disease_9.png
✅ Grad-CAM Visualization Completed!
✅ Class mapping file saved at: /home/srivatsa/Multiple_Crop_Disease_Detection/reports/Baseline_Model/Plots/GradCAM//class_mapping.txt


2025-03-24 18:29:39.814102: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
